In [1]:
import time
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
import os
import re
import pandas as pd
import numpy as np
from IPython.display import Audio
import seaborn as sns

import helpers # this is where the main training/decoding functions are, modified from teh original HIVAE main.py

#import warnings 
#warnings.filterwarnings('ignore') ########## NOTE: comment out for testing in case it's hiding problems

def set_settings(opts,nepochs=500,modload=False,save=True): # note: modload doesnt do anything right now, hardcoded in helpers.py
    'replace setting template placeholders with file info'
    inputf=re.sub('.csv','',opts['files'].iloc[0])
    missf=inputf+'_missing.csv'
    typef=inputf+'_types.csv'
    
    template = '--epochs NEPOCHS --model_name model_HIVAE_inputDropout --restore MODLOAD \
        --data_file data_python/INPUT_FILE.csv --types_file data_python/TYPES_FILE \
         --batch_size NBATCH --save NEPFILL --save_file SAVE_FILE\
        --dim_latent_s SDIM --dim_latent_z 1 --dim_latent_y YDIM \
        --miss_percentage_train 0 --miss_percentage_test 0 \
        --true_miss_file data_python/MISS_FILE --learning_rate LRATE'
    
    # replace placeholders in template
    settings = re.sub('INPUT_FILE',inputf,template)
    settings = re.sub('NBATCH',str(opts['nbatch'].iloc[0]),settings)
    settings = re.sub('NEPOCHS',str(nepochs),settings)
    settings = re.sub('NEPFILL',str(nepochs-1),settings) if save else re.sub('NEPFILL',str(nepochs*2),settings)
    settings = re.sub('YDIM',str(opts['ydims'].iloc[0]),settings)
    settings = re.sub('SDIM',str(opts['sdims'].iloc[0]),settings)
    settings = re.sub('MISS_FILE',missf,settings) #if not 'medhist' in inputf else re.sub('--true_miss_file data_python/MISS_FILE','',settings)
    settings = re.sub('TYPES_FILE',typef,settings)
    settings = re.sub('SAVE_FILE',inputf,settings)
    settings = re.sub('LRATE',str(opts['lrates'].iloc[0]),settings)
    settings = re.sub('MODLOAD','1',settings) if modload else re.sub('MODLOAD','0',settings)
    
    return settings

In [12]:
files=[i for i in os.listdir('data_python/') if not '_types' in i and not '_missing' in i and i and not '.DS_Store' in i]
files.sort()
print(files)

['ADAM_Metallopeptidase_subgraph.csv', 'AD_T2DM_SNPs.csv', 'APOE_subgraph.csv', 'ATP_binding_cassette_transport_subgraph.csv', 'Acetylcholine_signaling_subgraph.csv', 'Albumin_subgraph.csv', 'Alpha_2_macroglobulin_subgraph.csv', 'Amyloidogenic_subgraph.csv', 'Androgen_subgraph.csv', 'Apoptosis_signaling_subgraph.csv', 'Axonal_guidance_subgraph.csv', 'Axonal_transport_subgraph.csv', 'Bcl-2_subgraph.csv', 'Beta_secretase_subgraph.csv', 'Binding_and_Uptake_of_Ligands_by_Scavenger_Receptors.csv', 'Calcium-dependent_signal_transduction.csv', 'csf_VIS1.csv', 'volume_VIS1.csv', 'volume_VIS12.csv', 'volume_VIS24.csv', 'volume_VIS6.csv']


In [5]:
best_hyper = pd.read_csv('best_hyper.csv')

In [6]:
best_hyper

,lrates,nbatch,wdecay,ydims,files,loss,sdims
0,0.010,32,0.001,1,ADAM_Metallopeptidase_subgraph.csv,3.495601,3
1,0.010,16,0.000,1,AD_T2DM_SNPs.csv,0.884013,1
2,0.010,16,0.000,1,APOE_subgraph.csv,1.353774,1
3,0.010,32,0.000,1,ATP_binding_cassette_transport_subgraph.csv,31.059325,2
4,0.010,32,0.010,1,Acetylcholine_signaling_subgraph.csv,9.354569,1
5,0.010,32,0.001,1,Albumin_subgraph.csv,1.709542,1
6,0.010,16,0.001,1,Alpha_2_macroglobulin_subgraph.csv,0.724073,2
7,0.010,32,0.000,1,Amyloidogenic_subgraph.csv,2.283330,1
8,0.010,16,0.000,1,Androgen_subgraph.csv,0.289746,1
9,0.010,32,0.000,1,Apoptosis_signaling_subgraph.csv,11.163709,2


In [8]:
meta = pd.read_csv('metaenc.csv')
sample_size = 250
recon=list()
recdfs=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=meta['zcode_'+re.sub('.csv','',f)]
    scodes=meta['scode_'+re.sub('.csv','',f)]
    rec=helpers.dec_network(settings,zcodes,scodes)
    recon.append(rec)
    
    subj=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_subj.csv')['x']
    names=pd.read_csv('python_names/'+re.sub('.csv','',f)+'_cols.csv')['x']
    recd=pd.DataFrame(rec)
    recd.columns=names
    recd['SUBJID']=subj
    recdfs.append(recd)
    
recon_dic=dict(zip(files,recon))
data_recon=helpers.merge_dat(recdfs)
data_recon.to_csv('reconRP_test.csv',index=False)

/Users/msood/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Namespace(batch_size=250, data_file='data_python/ADAM_Metallopeptidase_subgraph.csv', dim_latent_s=3, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.01, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='ADAM_Metallopeptidase_subgraph', true_miss_file='data_python/ADAM_Metallopeptidase_subgraph_missing.csv', types_file='data_python/ADAM_Metallopeptidase_subgraph_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
Meemansa
types_dict
[{'type': 'ordinal', 'dim': '3', 'nclass': '3'}, {'type': 'ordinal', 'dim': '3', 'nclass': '3'}, {'type': 'ordinal', 'dim': '3', 'nclass': '3'}, {'type': 'ordinal', 'dim': '3', 'nclass': '3'}, {'type': 'ordinal', 'dim': '3', 'nclass': '3'}, {'type': 'ordinal', 'dim': '3', 'nclass': '

InvalidArgumentError: Restoring from checkpoint failed. This is most likely due to a mismatch between the current graph and the graph from the checkpoint. Please ensure that you have not altered the graph expected based on the checkpoint. Original error:

Assign requires shapes of both tensors to match. lhs shape= [3,1] rhs shape= [1,1]
	 [[node save/Assign_18 (defined at /Users/msood/Library/Python/3.7/lib/python/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'save/Assign_18':
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/msood/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/msood/Library/Python/3.7/lib/python/site-pack